In [476]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import pickle
import os
from glob import glob
from parse import parse
%matplotlib inline
import tensorflow as tf
import random
import tensorflow.contrib.layers as lays
import tensorflow.contrib.slim as slim
import multiprocessing
import itertools

parsedlisttestSing=pd.read_csv('parsedlisttestSing.csv')
parsedlisttrainSing=pd.read_csv('parsedlisttrainSing.csv')

## TO DO FIX THE HARDOCODED IMAGE RESIZE Values
def convert_to_image(path, system='sing', noise = True, varsM=['MeanDoCA','Mu', 'SmoothedNumPoCAs']):
    df = pd.read_csv(path)
    #df = df[df['z']>300]
    nx = len(df.x.unique())
    ny = len(df.y.unique())
    nz = len(df.z.unique())
    #df = df.sort_values(by=['z','y','x'])
    #image = df[varsM].values.reshape(-1,nx,ny,nz)
    ## normalize
    #df[varsM] = (df[varsM] - df[varsM].min())/(df[varsM].max()-df[varsM].min())
    df[varsM] = (df[varsM] - df[varsM].min())/(df[varsM].std())
    if noise:
        df[varsM] = df[varsM].apply(lambda x: x + 3*np.random.rand()/1)
    image = df[varsM].to_records(index=False).reshape(nx,ny,nz)
    image = np.array(image.tolist())
    # reshape the volume so the deconvolutio would work
    image = pad_along_axis(image,12,axis=0)
    image = pad_along_axis(image,12,axis=2)
    image = np.delete(image, 1, axis=1)
    #df=df.loc[df['object_names'] <>''] #not sure about this
    if system=='hmt_apg':
        listofMaterials = ['ClutterA','ANFO','ClutterC','AN','Sugar', 'ClutterB','PCFO']
    if system=='sing':
        listofMaterials = ['Cigarettes', 'PPC', 'SodiumNitrate', 'WheatFlour', 'Textiles']
    materialsPresent = df.columns.intersection(listofMaterials)
    if len(materialsPresent) >0:
        #df['AirOrNot'] = df[materialsPresent].apply(lambda row: 1 if sum(row)<=0.00 else 0, axis=1)
        df['AirOrNot'] = df[materialsPresent].apply(lambda row: 1 if sum(row)<=0.0 else 
                                                    2 if sum(row)>0.0 and sum(row)<=0.2 else 0 , axis=1)
        #df['AirOrNot'] = df[materialsPresent].apply(lambda row: 1 - sum(row), axis=1)
    else:
        df['AirOrNot'] = 1
    nxn = 12
    nyn = 24
    nzn = 12
    annotations = df['AirOrNot'].values.reshape(nx,ny,nz)
    annotations = pad_along_axis(annotations,12,axis=0)
    annotations = pad_along_axis(annotations,12,axis=2)
    annotations = np.delete(annotations, 1, axis=1)
    if noise:
        positions = random.randint(-1000,1000)
        image = np.roll(image, positions, axis=0)
        annotations = np.roll(annotations, positions, axis=0)
    return image, annotations

def get_batches_fn(batch_size, image_path,x, y, z, shuffle = True, varsM=['MeanDoCA','Mu', 'SmoothedNumPoCAs'], 
                   trainflag='Train', system='sing', eval_phase=False, datasettype='ds'):
    '''
    Create batches, restricting the number of scans per run included. 
    sample was on 2 scans per run, changed to 10 
    '''
    if eval_phase==False:
        m = image_path.groupby('run', group_keys=False).apply(lambda x: x.sample(min(len(x), 150)))
        if shuffle:
            shuffled = random.sample(m['filepath'].values, batch_size)
        else:
            shuffled = image_path['filepath'].values
        images = []
        labels = []
        for im_path in shuffled:
            if trainflag=='Train':
                image, anno = convert_to_image(im_path,system, True, varsM)
            else:
                image, anno = convert_to_image(im_path,system,False, varsM)
            images.append(image)
            labels.append(anno)
        labels = np.array(labels).reshape(batch_size, x*y*z)
        images = np.array(images)
        return tf.convert_to_tensor(images, dtype = tf.float32), tf.convert_to_tensor(labels, dtype = tf.float32)
    if eval_phase ==True and datasettype=='vtp':
        images = []
        for im_path in image_path:
            df = vtp_to_df(im_path)
            nx = len(df.x.unique())
            ny = len(df.y.unique())
            nz = len(df.z.unique())
            df[varsM] = (df[varsM] - df[varsM].min())/(df[varsM].max()-df[varsM].min())
            image = df[varsM].to_records(index=False).reshape(nx,ny,nz)
            image = np.array(image.tolist())
            # reshape the volume so the deconvolution would work
            image = pad_along_axis(image,12,axis=0)
            image = pad_along_axis(image,12,axis=2)
            image = np.delete(image, 0, axis=1)
            images.append(image)
        images = np.array(images)
        return tf.convert_to_tensor(images, dtype = tf.float32)
    if eval_phase ==True and datasettype=='ds':
        images = []
        labels = []
        for im_path in image_path:
            image, anno = convert_to_image(im_path,system,False, varsM)
            images.append(image)
            labels.append(anno)
        labels = np.array(labels).reshape(1,12,24,12)
        images = np.array(images)
        return tf.convert_to_tensor(images, dtype = tf.float32), tf.convert_to_tensor(labels, dtype = tf.float32)

In [430]:
def pad_along_axis(array, target_length, axis=0):
    pad_size = target_length - array.shape[axis]
    axis_nb = len(array.shape)
    if pad_size < 0:
        return a
    npad = [(0, 0) for x in range(axis_nb)]
    npad[axis] = (0, pad_size)
    b = np.pad(array, pad_width=npad, mode='constant', constant_values=1)
    return b

In [523]:
# build the graph
x = 12#22
y = 24#46
z = 12 #9 #18 #22
batch_size =50
graph2 = tf.Graph()

#varsM=['StoppedFracP', 'StoppedMean','FOM', 'Mu', 'box_p25', 'box_p50', 'dist_1_2']
#varsM = ['FracMu', 'GeoMeanPThetaSquared', 'MikesFOM', 'Mu', 'box_p95', 'value_max_1', 'SmoothedNumPoCAs']
#varsM = ['FracMu', 'GeoMeanPThetaSquared', 'Mu', 'box_p95', 'value_max_1']
#varsM = ['GeoMeanPThetaSquared'constant, 'NumTracks', 'box_mean',  'box_p95', 'value_max_1', 'sum_max_1_5', 'NumStoppedTracks', 'box_std']
#varsM = ['dist_1_2', 'dist_1_3', 'dist_2_3', 'n_2sigma', 'n_3sigma']
#varsM = ['FracMu', 'box_p95','ScatteringRatio', 'n_3sigma','StoppedMeanP', 'SmoothedNumPoCAs', 
#         'StoppedThroughRatio', 'PoCAScat'] #<- TF7

varsM = ['FracMu', 'box_p95','ScatteringRatio', 'n_3sigma','StoppedMeanP', 'SmoothedNumPoCAs', 
         'StoppedThroughRatio', 'PoCAScat']
#varsM = ['box_p95','ScatteringRatio','GeoMeanTheta']


# 'dist_1_3', 'dist_2_3', 'n_2sigma', 'sum_max_2_2', 'sum_max_3_2','sum_max_3_3', 'n_3sigma', 'FOMFrac', 
''' "box_mean", "box_p25", "box_p50", "box_p75" , "box_p95", "box_std", "dist_1_2", 
"dist_1_3", "dist_2_3", "n_1sigma", "n_2sigma" , "n_3sigma", "sum_max_1_2", "sum_max_1_3",
"sum_max_1_5", "sum_max_2_2", "sum_max_2_3", "sum_max_2_5", "sum_max_3_2", "sum_max_3_3", "sum_max_3_5" , 
"value_max_1", "value_max_2", "value_max_3",  "SmoothedNumPoCAs", "LogPTheta", "LogPThetaSquared", 
"GeoMeanPThetaSquared", "Sigma", "MeanTheta", "GeoMeanTheta", "NumTracks", "Mu", "PoCARate", 
"MeanDoCA", "GeoMeanFractionalScattering" , "MeanFractionalScattering" , 
"ScatteringRatio", "FracMu", "FracSigma", "FOMFrac", "FOM", "MikesFOM", "PoCAScat", "PoCALogScat", 
"NumStoppedTracks", "StoppedMean", "StoppedMeanP", 
"StoppedFracP", "StoppedThroughRatio",
 "x_max_1", "y_max_1",  "z_max_1"] 
 
'''
total_batch_train = len(parsedlisttrainSing)
total_batch_test = len(parsedlisttestSing)
with graph2.as_default():
        # Placeholders
    #batch_size = tf.placeholder(tf.float32, name='batch_size')    
    input_images = tf.placeholder(tf.float32, shape=(None, x, y, z, (len(varsM))), name='input_images')
    input_labels = tf.placeholder(tf.float32, shape=(None, x*y*z), name='input_labels')
    phase = tf.placeholder(tf.bool, name='phase')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    eval_phase = tf.placeholder(tf.bool, name='eval_phase')

    # Global step
    global_step = tf.Variable(0, trainable=False)
    datatopulltrain = parsedlisttrainSing[parsedlisttrainSing['secs']>=120]#parsedlisttrain.sample(total_batch_test)
    datatopulltest = parsedlisttestSing[parsedlisttestSing['secs']>=120]#parsedlisttest.sample(total_batch_test)
    ## Test and Train flag control if noise is added to the volume
    eval_images, eval_labels = get_batches_fn(batch_size, datatopulltest, x, y, z, True, varsM, 'Test','sing', False,'ds')
    train_images, train_labels = get_batches_fn(batch_size, datatopulltrain, x, y, z, True, varsM, 'Test', 'sing', False,'ds')
    #eval_image_no_shuffle, eval_labels_no_shuffle = get_eval_im(datatopulltest[0:1], x, y, z, varsM)
    
    def shape_so(tensor):
        s = tensor.get_shape()
        return tuple([s[i].value for i in range(0, len(s))])
    
    def fcn_v3(inputs, x,y,z, phase):
        # encoder
        shape_to_code = shape_so(inputs)
        print(shape_to_code)
        #was 6
        net = lays.conv3d(inputs, 6*len(varsM), [1, 1, 1], stride=1, padding='SAME', activation_fn=tf.nn.leaky_relu)
        net = lays.conv3d(net, 6*len(varsM), [1, 1, 1], stride=1, padding='SAME', activation_fn=tf.nn.leaky_relu)
        net = lays.conv3d(net, 6*len(varsM), [2,2,2], stride=2, padding='SAME', activation_fn=tf.nn.leaky_relu)        
        net = tf.contrib.layers.batch_norm(net,center=True, scale=True, is_training=phase )
        net = tf.nn.dropout(net, keep_prob)
        shape_to_code = shape_so(net)
        print('First Layer {}',shape_to_code)
        # was 10
        net = lays.conv3d(net, 3*len(varsM), [1, 1, 1], stride=1, padding='SAME', activation_fn=tf.nn.leaky_relu)
        net = lays.conv3d(net, 3*len(varsM), [2,2,2], stride=2, padding='SAME', activation_fn=tf.nn.leaky_relu)
        net = tf.contrib.layers.batch_norm(net,center=True, scale=True, is_training=phase )
        net = tf.nn.dropout(net, keep_prob)
        shape_to_code = shape_so(net)
        print('Second Layer {}',shape_to_code)        
        #x and z get from 3 to 2 if this is at stride 2, and then I cannot redo the convolution
        #net = lays.conv3d(net, 1*len(varsM), [1, 1, 1], stride=1, padding='SAME', activation_fn=tf.nn.leaky_relu)
        #net = lays.conv3d(net, 1*len(varsM), [2,2,2], stride=2, padding='SAME', activation_fn=tf.nn.leaky_relu) 
        #shape_to_code = shape_so(net)
        #print('Third Layer {}',shape_to_code)

        net = tf.layers.conv3d_transpose(net, 3*len(varsM), [1, 1, 1], strides=[1, 1, 1], padding='SAME', activation=tf.nn.leaky_relu)
        net = tf.layers.conv3d_transpose(net, 3*len(varsM), [2,2,2], strides=[2,2,2], padding='SAME', activation=tf.nn.leaky_relu)
        net = tf.contrib.layers.batch_norm(net,center=True, scale=True, is_training=phase )
        net = tf.nn.dropout(net, keep_prob)

        shape_to_code = shape_so(net)
        print('First Deconv Layer {}',shape_to_code)
        
        net = tf.layers.conv3d_transpose(net, 6*len(varsM), [1, 1, 1], strides=[1, 1, 1], padding='SAME', activation=tf.nn.leaky_relu)
        net = tf.layers.conv3d_transpose(net, 6*len(varsM), [1, 1, 1], strides=[1, 1, 1], padding='SAME', activation=tf.nn.leaky_relu)
        net = tf.layers.conv3d_transpose(net, 6*len(varsM), [2,2,2], strides=[2, 2, 2], padding='SAME', activation=tf.nn.leaky_relu)
        #    
        net = tf.contrib.layers.batch_norm(net,center=True, scale=True, is_training=phase )
        net = tf.nn.dropout(net, keep_prob)       
        
        shape_to_code = shape_so(net)
        print('Second Deconv Layer {}',shape_to_code)
        
        #final layer
        #net = lays.conv3d(net, 1*len(varsM), [1, 1, 1], stride=1, padding='SAME', activation_fn=tf.nn.leaky_relu)
        net = tf.layers.conv3d_transpose(net, 3, [1, 1, 1], strides=[1, 1, 1], padding='SAME'
                                         , activation=tf.nn.leaky_relu)
     
        #net = tf.contrib.layers.batch_norm(net,center=True, scale=True, is_training=phase )
 
        # fully connected layers
        #net = tf.layers.flatten(net)
        net = tf.reshape(net, [-1,3,x*y*z])
        shape_to_code = shape_so(net)
        print('Final Net {}', shape_to_code)
        return net     

    fcn_outputs = tf.reshape(tensor=fcn_v3(input_images, x,y,z, phase),shape=(-1,x*y*z,3) )
    predictionsOut = tf.argmax(fcn_outputs, dimension=2, name='fcn_outputs')
    
    labelsOH = tf.one_hot(indices=tf.cast(input_labels, tf.int32), depth=3)
    print('Shape of Labels {}', shape_so(labelsOH))
    print('Shape of Predictions {}', shape_so(predictionsOut))
    loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(logits=fcn_outputs, labels=labelsOH))
    MIoU, mIoUop = tf.metrics.mean_iou(predictions=tf.cast(tf.round(predictionsOut),tf.int32),
                                       labels=tf.cast(tf.round(input_labels),tf.int32),  num_classes=3,
                                      name="mean_union")
    is_correct = tf.equal(tf.cast(tf.round(predictionsOut), tf.int64), tf.cast(input_labels, tf.int64))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

    # Training operations
    learning_rate = tf.train.exponential_decay(learning_rate=0.00001,#0.00001, #0.001
                                               global_step=global_step, 
                                               #decay_steps=int(train_images.shape[0] / (2 * batch_size)), 
                                               decay_steps = 1000,
                                               decay_rate=0.95, 
                                               staircase=True)
    
    #trainer = tf.train.RMSPropOptimizer(learning_rate)
    trainer = tf.train.AdamOptimizer(learning_rate)
    training_step = trainer.minimize(loss)
    saver = tf.train.Saver()
    
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8, allow_growth=False)
#config = tf.ConfigProto(gpu_options=gpu_options,allow_soft_placement=True)

config = tf.ConfigProto(intra_op_parallelism_threads=multiprocessing.cpu_count(), 
                        inter_op_parallelism_threads=multiprocessing.cpu_count(), 
                        log_device_placement=True,
                        gpu_options=gpu_options,
                        allow_soft_placement=True, 
                        device_count = {'CPU' : 8, 
                                        'GPU' : 1}
                       )
        
session2 = tf.InteractiveSession(graph=graph2, config=config)
tf.global_variables_initializer().run()
tf.local_variables_initializer().run()
tf.train.start_queue_runners(sess=session2, start = True)
with tf.control_dependencies([mIoUop]):
    MIoU = tf.identity(MIoU)


(None, 12, 24, 12, 8)
('First Layer {}', (None, 6, 12, 6, 12))
('Second Layer {}', (None, 3, 6, 3, 6))
('First Deconv Layer {}', (None, 6, 12, 6, 6))
('Second Deconv Layer {}', (None, 12, 24, 12, 12))
('Final Net {}', (None, 3, 3456))
('Shape of Labels {}', (None, 3456, 3))
('Shape of Predictions {}', (None, 3456))


In [524]:
for e in range(550):
    total_train_loss = 0.0
    total_test_loss = 0.0
    total_fn = 0.0
    total_fnt = 0.0
    total_miou = 0.0
    total_miout = 0.0
    test_loss = 0
    test_IoU = 0
    total_batch = 1*int(total_batch_train/batch_size)
    for i in range(total_batch):
        #session2.run(running_vars_initializer)
        train_x, train_y = session2.run([train_images, train_labels])
        test_x, test_y = session2.run([eval_images, eval_labels])
        #_, train_loss = session2.run([training_step, loss], feed_dict={input_images: train_x, input_labels: train_y})
        #_ = session2.run([], feed_dict={input_images: train_x, input_labels: train_y, 
        #                                             phase:'True', keep_prob:'0.35'})
        _, train_loss,acc = session2.run([training_step, loss,accuracy], feed_dict={input_images: train_x, 
                                                               input_labels: train_y, phase:'True', eval_phase:False,
                                                                keep_prob:'0.50'})
        #if i % 10 ==0:
        _, test_loss, acc1= session2.run([predictionsOut, loss, accuracy], feed_dict={input_images: test_x, 
                                                                              input_labels: test_y, eval_phase:False,
                                                              phase:'False', keep_prob:'1.0'})
        #session2.run([mIoUop], feed_dict={input_images: test_x, input_labels: test_y, phase:'False', keep_prob:'1.0'})
        total_train_loss += train_loss
        total_test_loss += test_loss
        total_fn +=acc
        total_fnt +=acc1
        miout = session2.run([MIoU], feed_dict={input_images: test_x, input_labels: test_y, 
                                                              phase:'False', keep_prob:'1.0'}) 
        
        total_miout +=miout[0]
        total_miou +=0
        #print('batch {} - Train loss: {} - Test loss: {} - Test FN {}'.format(i+1,fn, test_loss, fnt)) - TrainRatio {} -TestRatio {}
    print('Epoch {} - Train loss: {} -Test loss: {} -Diff {} -Train FN {} -Test FN {} -Train MIoU {} -Test MIou {}'.format(e+1, np.round(total_train_loss/total_batch,3), np.round(total_test_loss/total_batch,3),
                 np.round(total_test_loss/total_batch,3) - np.round(total_train_loss/total_batch,3),
                 np.round(total_fn/total_batch,2), np.round(total_fnt/total_batch,3), 
                 np.round(total_miou/total_batch,3), np.round(total_miout/total_batch,3)))

Epoch 1 - Train loss: 218288.248 -Test loss: 210401.743 -Diff -7886.505 -Train FN 0.21 -Test FN 0.2 -Train MIoU 0.0 -Test MIou 0.071
Epoch 2 - Train loss: 213463.678 -Test loss: 204908.145 -Diff -8555.533 -Train FN 0.22 -Test FN 0.203 -Train MIoU 0.0 -Test MIou 0.072
Epoch 3 - Train loss: 209033.581 -Test loss: 199848.291 -Diff -9185.29 -Train FN 0.23 -Test FN 0.213 -Train MIoU 0.0 -Test MIou 0.073
Epoch 4 - Train loss: 204856.985 -Test loss: 195087.595 -Diff -9769.39 -Train FN 0.24 -Test FN 0.23 -Train MIoU 0.0 -Test MIou 0.074
Epoch 5 - Train loss: 200863.287 -Test loss: 190745.033 -Diff -10118.254 -Train FN 0.25 -Test FN 0.254 -Train MIoU 0.0 -Test MIou 0.077
Epoch 6 - Train loss: 197130.432 -Test loss: 186843.236 -Diff -10287.196 -Train FN 0.27 -Test FN 0.281 -Train MIoU 0.0 -Test MIou 0.08
Epoch 7 - Train loss: 193634.134 -Test loss: 183287.297 -Diff -10346.837 -Train FN 0.29 -Test FN 0.319 -Train MIoU 0.0 -Test MIou 0.084
Epoch 8 - Train loss: 190480.568 -Test loss: 179995.426 -D

Epoch 62 - Train loss: 116037.047 -Test loss: 106580.54 -Diff -9456.507 -Train FN 0.64 -Test FN 0.616 -Train MIoU 0.0 -Test MIou 0.19
Epoch 63 - Train loss: 115467.099 -Test loss: 105991.73 -Diff -9475.369 -Train FN 0.64 -Test FN 0.616 -Train MIoU 0.0 -Test MIou 0.191
Epoch 64 - Train loss: 114846.944 -Test loss: 105390.264 -Diff -9456.68 -Train FN 0.64 -Test FN 0.616 -Train MIoU 0.0 -Test MIou 0.191
Epoch 65 - Train loss: 114235.799 -Test loss: 104781.659 -Diff -9454.14 -Train FN 0.65 -Test FN 0.616 -Train MIoU 0.0 -Test MIou 0.192
Epoch 66 - Train loss: 113639.847 -Test loss: 104165.546 -Diff -9474.301 -Train FN 0.65 -Test FN 0.616 -Train MIoU 0.0 -Test MIou 0.192
Epoch 67 - Train loss: 113006.172 -Test loss: 103548.1 -Diff -9458.072 -Train FN 0.66 -Test FN 0.616 -Train MIoU 0.0 -Test MIou 0.192
Epoch 68 - Train loss: 112359.667 -Test loss: 102936.509 -Diff -9423.158 -Train FN 0.67 -Test FN 0.616 -Train MIoU 0.0 -Test MIou 0.192
Epoch 69 - Train loss: 111631.172 -Test loss: 102330.87

Epoch 123 - Train loss: 48812.47 -Test loss: 36520.826 -Diff -12291.644 -Train FN 0.96 -Test FN 0.965 -Train MIoU 0.0 -Test MIou 0.248
Epoch 124 - Train loss: 48173.022 -Test loss: 35884.517 -Diff -12288.505 -Train FN 0.96 -Test FN 0.965 -Train MIoU 0.0 -Test MIou 0.249
Epoch 125 - Train loss: 47558.596 -Test loss: 35284.328 -Diff -12274.268 -Train FN 0.96 -Test FN 0.966 -Train MIoU 0.0 -Test MIou 0.25
Epoch 126 - Train loss: 46966.708 -Test loss: 34692.993 -Diff -12273.715 -Train FN 0.96 -Test FN 0.966 -Train MIoU 0.0 -Test MIou 0.251
Epoch 127 - Train loss: 46333.417 -Test loss: 34100.465 -Diff -12232.952 -Train FN 0.96 -Test FN 0.967 -Train MIoU 0.0 -Test MIou 0.252
Epoch 128 - Train loss: 45742.781 -Test loss: 33537.761 -Diff -12205.02 -Train FN 0.96 -Test FN 0.968 -Train MIoU 0.0 -Test MIou 0.253
Epoch 129 - Train loss: 45178.159 -Test loss: 32985.622 -Diff -12192.537 -Train FN 0.96 -Test FN 0.968 -Train MIoU 0.0 -Test MIou 0.254
Epoch 130 - Train loss: 44634.413 -Test loss: 32446

Epoch 184 - Train loss: 26539.715 -Test loss: 15757.32 -Diff -10782.395 -Train FN 0.97 -Test FN 0.98 -Train MIoU 0.0 -Test MIou 0.293
Epoch 185 - Train loss: 26390.91 -Test loss: 15623.212 -Diff -10767.698 -Train FN 0.97 -Test FN 0.98 -Train MIoU 0.0 -Test MIou 0.294
Epoch 186 - Train loss: 26206.202 -Test loss: 15508.784 -Diff -10697.418 -Train FN 0.97 -Test FN 0.98 -Train MIoU 0.0 -Test MIou 0.294
Epoch 187 - Train loss: 26044.726 -Test loss: 15376.631 -Diff -10668.095 -Train FN 0.97 -Test FN 0.98 -Train MIoU 0.0 -Test MIou 0.295
Epoch 188 - Train loss: 25869.926 -Test loss: 15275.584 -Diff -10594.342 -Train FN 0.97 -Test FN 0.98 -Train MIoU 0.0 -Test MIou 0.295
Epoch 189 - Train loss: 25736.872 -Test loss: 15160.496 -Diff -10576.376 -Train FN 0.97 -Test FN 0.98 -Train MIoU 0.0 -Test MIou 0.296
Epoch 190 - Train loss: 25596.193 -Test loss: 15040.207 -Diff -10555.986 -Train FN 0.97 -Test FN 0.98 -Train MIoU 0.0 -Test MIou 0.296
Epoch 191 - Train loss: 25397.724 -Test loss: 14959.348 -

Epoch 245 - Train loss: 20672.914 -Test loss: 12615.983 -Diff -8056.931 -Train FN 0.97 -Test FN 0.979 -Train MIoU 0.0 -Test MIou 0.32
Epoch 246 - Train loss: 20629.115 -Test loss: 12592.382 -Diff -8036.733 -Train FN 0.97 -Test FN 0.979 -Train MIoU 0.0 -Test MIou 0.32
Epoch 247 - Train loss: 20557.148 -Test loss: 12598.036 -Diff -7959.112 -Train FN 0.97 -Test FN 0.979 -Train MIoU 0.0 -Test MIou 0.32
Epoch 248 - Train loss: 20563.804 -Test loss: 12606.964 -Diff -7956.84 -Train FN 0.97 -Test FN 0.979 -Train MIoU 0.0 -Test MIou 0.321
Epoch 249 - Train loss: 20475.428 -Test loss: 12546.189 -Diff -7929.239 -Train FN 0.97 -Test FN 0.979 -Train MIoU 0.0 -Test MIou 0.321
Epoch 250 - Train loss: 20452.768 -Test loss: 12526.01 -Diff -7926.758 -Train FN 0.97 -Test FN 0.979 -Train MIoU 0.0 -Test MIou 0.321
Epoch 251 - Train loss: 20405.81 -Test loss: 12513.063 -Diff -7892.747 -Train FN 0.97 -Test FN 0.979 -Train MIoU 0.0 -Test MIou 0.322
Epoch 252 - Train loss: 20319.238 -Test loss: 12504.707 -Diff

Epoch 307 - Train loss: 18550.804 -Test loss: 12279.585 -Diff -6271.219 -Train FN 0.97 -Test FN 0.98 -Train MIoU 0.0 -Test MIou 0.337
Epoch 308 - Train loss: 18553.439 -Test loss: 12200.696 -Diff -6352.743 -Train FN 0.97 -Test FN 0.98 -Train MIoU 0.0 -Test MIou 0.337
Epoch 309 - Train loss: 18535.267 -Test loss: 12232.592 -Diff -6302.675 -Train FN 0.97 -Test FN 0.98 -Train MIoU 0.0 -Test MIou 0.337
Epoch 310 - Train loss: 18497.559 -Test loss: 12228.34 -Diff -6269.219 -Train FN 0.97 -Test FN 0.98 -Train MIoU 0.0 -Test MIou 0.338
Epoch 311 - Train loss: 18486.904 -Test loss: 12246.959 -Diff -6239.945 -Train FN 0.97 -Test FN 0.98 -Train MIoU 0.0 -Test MIou 0.338
Epoch 312 - Train loss: 18481.537 -Test loss: 12278.8 -Diff -6202.737 -Train FN 0.97 -Test FN 0.98 -Train MIoU 0.0 -Test MIou 0.338
Epoch 313 - Train loss: 18486.971 -Test loss: 12273.707 -Diff -6213.264 -Train FN 0.97 -Test FN 0.98 -Train MIoU 0.0 -Test MIou 0.338
Epoch 314 - Train loss: 18419.285 -Test loss: 12262.249 -Diff -61

Epoch 369 - Train loss: 17437.176 -Test loss: 12392.101 -Diff -5045.075 -Train FN 0.97 -Test FN 0.978 -Train MIoU 0.0 -Test MIou 0.349
Epoch 370 - Train loss: 17457.777 -Test loss: 12390.712 -Diff -5067.065 -Train FN 0.97 -Test FN 0.978 -Train MIoU 0.0 -Test MIou 0.349
Epoch 371 - Train loss: 17428.848 -Test loss: 12423.5 -Diff -5005.348 -Train FN 0.97 -Test FN 0.978 -Train MIoU 0.0 -Test MIou 0.35
Epoch 372 - Train loss: 17437.523 -Test loss: 12468.61 -Diff -4968.913 -Train FN 0.97 -Test FN 0.978 -Train MIoU 0.0 -Test MIou 0.35
Epoch 373 - Train loss: 17407.6 -Test loss: 12429.809 -Diff -4977.791 -Train FN 0.97 -Test FN 0.978 -Train MIoU 0.0 -Test MIou 0.35
Epoch 374 - Train loss: 17406.971 -Test loss: 12425.437 -Diff -4981.534 -Train FN 0.97 -Test FN 0.978 -Train MIoU 0.0 -Test MIou 0.35
Epoch 375 - Train loss: 17390.714 -Test loss: 12414.545 -Diff -4976.169 -Train FN 0.97 -Test FN 0.978 -Train MIoU 0.0 -Test MIou 0.35
Epoch 376 - Train loss: 17414.858 -Test loss: 12400.957 -Diff -50

Epoch 430 - Train loss: 16843.713 -Test loss: 12609.339 -Diff -4234.374 -Train FN 0.97 -Test FN 0.976 -Train MIoU 0.0 -Test MIou 0.358
Epoch 431 - Train loss: 16782.908 -Test loss: 12621.832 -Diff -4161.076 -Train FN 0.97 -Test FN 0.976 -Train MIoU 0.0 -Test MIou 0.358
Epoch 432 - Train loss: 16784.415 -Test loss: 12580.075 -Diff -4204.34 -Train FN 0.97 -Test FN 0.976 -Train MIoU 0.0 -Test MIou 0.358
Epoch 433 - Train loss: 16767.353 -Test loss: 12597.799 -Diff -4169.554 -Train FN 0.97 -Test FN 0.976 -Train MIoU 0.0 -Test MIou 0.358
Epoch 434 - Train loss: 16764.409 -Test loss: 12632.43 -Diff -4131.979 -Train FN 0.97 -Test FN 0.976 -Train MIoU 0.0 -Test MIou 0.359
Epoch 435 - Train loss: 16749.745 -Test loss: 12617.705 -Diff -4132.04 -Train FN 0.97 -Test FN 0.976 -Train MIoU 0.0 -Test MIou 0.359
Epoch 436 - Train loss: 16747.017 -Test loss: 12585.681 -Diff -4161.336 -Train FN 0.97 -Test FN 0.976 -Train MIoU 0.0 -Test MIou 0.359
Epoch 437 - Train loss: 16746.644 -Test loss: 12607.332 -D

Epoch 491 - Train loss: 16340.336 -Test loss: 12618.512 -Diff -3721.824 -Train FN 0.97 -Test FN 0.974 -Train MIoU 0.0 -Test MIou 0.365
Epoch 492 - Train loss: 16378.314 -Test loss: 12637.254 -Diff -3741.06 -Train FN 0.97 -Test FN 0.974 -Train MIoU 0.0 -Test MIou 0.365
Epoch 493 - Train loss: 16342.114 -Test loss: 12627.835 -Diff -3714.279 -Train FN 0.97 -Test FN 0.974 -Train MIoU 0.0 -Test MIou 0.365
Epoch 494 - Train loss: 16306.821 -Test loss: 12619.519 -Diff -3687.302 -Train FN 0.97 -Test FN 0.974 -Train MIoU 0.0 -Test MIou 0.365
Epoch 495 - Train loss: 16329.589 -Test loss: 12660.809 -Diff -3668.78 -Train FN 0.97 -Test FN 0.974 -Train MIoU 0.0 -Test MIou 0.365
Epoch 496 - Train loss: 16338.133 -Test loss: 12643.687 -Diff -3694.446 -Train FN 0.97 -Test FN 0.974 -Train MIoU 0.0 -Test MIou 0.365
Epoch 497 - Train loss: 16301.32 -Test loss: 12656.567 -Diff -3644.753 -Train FN 0.97 -Test FN 0.974 -Train MIoU 0.0 -Test MIou 0.365
Epoch 498 - Train loss: 16300.754 -Test loss: 12612.436 -D

In [491]:
saver.save(session2, 'Documents/AirNoAirTF/TFSelectedSing_12_3Classes')

'Documents/AirNoAirTF/TFSelectedSing_12_3Classes'

In [526]:
eval_images1, labels = get_batches_fn(1, parsedlisttrainSing['filepath'][955:956], x, y, z, True, varsM, 'Test','sing', True, datasettype='ds')
test_x = session2.run(eval_images1)
#test_x1 = tf.reshape(test_x,[-1,x,y,z,len(varsM)])
pred1 = session2.run(predictionsOut, feed_dict={input_images: test_x, eval_phase:True,
                                                              phase:'False', keep_prob:'1.0'})
labels1 = np.reshape(labels.eval(),12*24*12)
manual_eval = pd.DataFrame({'prediction':pred1[0],'actual':labels1})
pd.crosstab(manual_eval.prediction, manual_eval.actual)

actual,0.0,1.0,2.0
prediction,,,
0,27,23,16
1,3,3385,2


In [414]:
parsedlisttrainSing['filepath'][651]

'/storage/opt/dsc/dse/sing/release/3.4.19/output/contraband_tagging_30cm_refit/output/run/8/600/0016/ContrabandFeatures_0_1_120.csv'

In [320]:
colors = ['rgb(0,0,255)', 'rgb(0,0,0)','rgb(0,0,255)', 'rgb(0,255,0)', 'rgb(0,128,255)', 'rgb(255,0,255)'
          , 'rgb(255,255,51)', 'rgb(255,229,209)', 'rgb(0,255,255)']
opacity_s = ['1','0.01','1','1','1','1','1','1','1']
data = []
plot_var = 'PredictedAir'
dfA=dfA.sort_values(by=['x','y','z'])
for i in range(len(dfA[plot_var].unique())):
    name = np.sort(dfA[plot_var].unique())[i]
    color = colors[i]
    opacity = opacity_s[i]
    x = dfA[ dfA[plot_var] == name ]['x']
    y = dfA[ dfA[plot_var] == name ]['y']
    z = dfA[ dfA[plot_var] == name ]['z']
    
    trace = dict(
        name = name,
        x = x, y = y, z = z,
        type = "scatter3d",    
        mode = 'markers',
        marker = dict( size=5, color=color, line=dict(width=0), opacity=opacity ) )
    data.append( trace )
    
layout = go.Layout(
    title='dfB[[Int1,Int3,Int2,Int4]]'
)

#plotly.offline.plot([trace, layout])
fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig)    

In [374]:
eval_images1

<tf.Tensor 'Const_8:0' shape=(1, 12, 24, 12, 8) dtype=float32>